In [233]:
import pandas as pd
import requests
import time
import os
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import seaborn as sns
from geopy.distance import geodesic
from sklearn.cluster import DBSCAN
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm
from geopy.distance import geodesic

#### Visu de l'historique des feux

In [234]:
path = 13
feux = pd.read_csv('https://projet-incendie.s3.eu-west-3.amazonaws.com/historique_incendies_avec_coordonnees.csv', sep=';', encoding='utf-8')
pd.set_option('display.max_columns', None)

feux.head(10)

,Département,Code INSEE,Nom de la commune,latitude,longitude,Date,Surface parcourue (m2),Surface forêt (m2),Surface maquis garrigues (m2),Autres surfaces naturelles hors forêt (m2),Surfaces agricoles (m2),Autres surfaces (m2),Surface autres terres boisées (m2),Surfaces non boisées naturelles (m2),Surfaces non boisées artificialisées (m2),Surfaces non boisées (m2),Type de peuplement,Nature,Feux
0,66,66190,Salses-le-Château,42.833380,2.918180,2014-01-02 14:56:00,10000,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Involontaire (particulier),1
1,06,06074,Lantosque,43.973468,7.312593,2014-01-07 13:50:00,48400,NaN,48400.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,1
2,2A,2A139,Lecci,41.679373,9.317798,2014-01-09 16:14:00,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,65,65173,Esterre,42.874901,0.006078,2014-01-10 20:04:00,10000,10000.0,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,NaN,NaN,1
4,2B,2B058,Canari,42.845933,9.331333,2014-01-11 14:00:00,100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
5,65,65233,Jarret,43.082300,-0.014354,2014-01-12 16:41:00,200,0.0,NaN,200.0,NaN,NaN,200.0,0.0,0.0,0.0,NaN,NaN,1
6,30,30051,Branoux-les-Taillades,44.219600,3.991240,2014-01-13 19:24:00,2500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
7,66,66196,Sorède,42.530402,2.956454,2014-01-17 16:40:00,10000,NaN,10000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,Involontaire (travaux),1
8,2B,2B242,Poggio-Mezzana,42.397680,9.493930,2014-01-18 14:57:00,3000,0.0,3000.0,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,NaN,1
9,974,97421,Salazie,-21.024383,55.543545,2014-01-21 14:08:00,3000,0.0,NaN,3000.0,NaN,NaN,3000.0,0.0,0.0,0.0,NaN,NaN,1


In [235]:
mask = feux['Département'] == f'{path}'
feux_ville = feux[mask]
feux_ville

,Département,Code INSEE,Nom de la commune,latitude,longitude,Date,Surface parcourue (m2),Surface forêt (m2),Surface maquis garrigues (m2),Autres surfaces naturelles hors forêt (m2),Surfaces agricoles (m2),Autres surfaces (m2),Surface autres terres boisées (m2),Surfaces non boisées naturelles (m2),Surfaces non boisées artificialisées (m2),Surfaces non boisées (m2),Type de peuplement,Nature,Feux
338,13,13002,Allauch,43.335138,5.483269,2014-04-06 17:18:00,10,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,1
379,13,13040,Fuveau,43.453910,5.560809,2014-04-10 17:46:00,50,20.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,Malveillance,1
405,13,13040,Fuveau,43.453910,5.560809,2014-04-14 16:42:00,100,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,Malveillance,1
406,13,13040,Fuveau,43.453910,5.560809,2014-04-14 16:42:00,100,50.0,50.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,Malveillance,1
408,13,13044,Grans,43.606758,5.063379,2014-04-14 18:10:00,20,20.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49917,13,13040,Fuveau,43.453910,5.560809,2013-09-26 00:31:00,100,100.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,Malveillance,1
49929,13,13040,Fuveau,43.453910,5.560809,2013-09-29 00:43:00,60,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,Malveillance,1
49930,13,13040,Fuveau,43.453910,5.560809,2013-09-29 06:08:00,6000,6000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,Malveillance,1
49999,13,13002,Allauch,43.335138,5.483269,2013-10-22 19:09:00,114000,114000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,Malveillance,1


#### Rajout de la météo

In [240]:

df_meteo=pd.read_csv(f'output/{path}_meteo_cleaned_features.csv', sep=';', encoding='utf-8')
df_meteo.head()

C:\Users\m_bar\AppData\Local\Temp\ipykernel_131148\1125191378.py:1: DtypeWarning: Columns (36) have mixed types. Specify dtype option on import or set low_memory=False.
  df_meteo=pd.read_csv(f'output/{path}_meteo_cleaned_features.csv', sep=';', encoding='utf-8')


,POSTE,DATE,RR,DRR,TN,HTN,TX,HTX,TM,TMNX,TNSOL,TN50,TAMPLI,TNTXM,FFM,FXI,DXI,HXI,FXY,DXY,HXY,FXI3S,HXI3S,UN,HUN,UX,HUX,DHUMI40,DHUMI80,TSVM,UM,ORAG,BRUME,ETPMON,ETPGRILLE,Code INSEE,Code Postal,ville,geo_point_2d,moyenne precipitations année,moyenne precipitations mois,moyenne evapotranspiration année,moyenne evapotranspiration mois,moyenne vitesse vent année,moyenne vitesse vent mois,moyenne temperature année,moyenne temperature mois
0,13001009,2006-01-01,0.2,NaN,3.7,823.0,10.2,1413.0,6.1,6.95,0.4,NaN,6.5,7.0,1.7,13.4,330.0,1157.0,6.3,320.0,1220.0,NaN,NaN,50.0,1432.0,98.0,7.0,0.0,1028.0,7.9,85.0,NaN,NaN,0.7,0.8,13001,13080/13090/13100/13290/13540,Aix-En-Provence,"43.53607083783732, 5.398574445818331",0.51,0.61,2.05,0.96,NaN,1.94,5.19,1.95
1,13001009,2006-01-02,0.0,NaN,2.8,604.0,9.9,1331.0,4.8,6.35,-0.9,NaN,7.1,6.4,2.5,12.8,330.0,1105.0,6.7,330.0,1107.0,NaN,NaN,57.0,1408.0,97.0,2348.0,0.0,814.0,6.7,79.0,NaN,NaN,1.2,1.2,13001,13080/13090/13100/13290/13540,Aix-En-Provence,"43.53607083783732, 5.398574445818331",0.51,0.75,2.07,0.95,NaN,1.91,5.23,1.96
2,13001009,2006-01-03,0.2,NaN,-2.6,610.0,10.1,1319.0,1.9,3.75,-7.2,NaN,12.7,3.8,0.8,6.4,330.0,1421.0,3.5,320.0,1430.0,NaN,NaN,39.0,1320.0,98.0,148.0,47.0,848.0,5.1,77.0,NaN,NaN,0.9,0.8,13001,13080/13090/13100/13290/13540,Aix-En-Provence,"43.53607083783732, 5.398574445818331",0.51,0.75,2.09,0.97,NaN,1.98,5.27,2.05
3,13001009,2006-01-04,0.0,NaN,-3.8,704.0,10.2,1247.0,1.2,3.20,-8.2,NaN,14.0,3.2,0.8,5.2,20.0,926.0,2.6,260.0,1348.0,NaN,NaN,40.0,1227.0,95.0,2152.0,6.0,856.0,4.9,77.0,NaN,NaN,0.9,0.7,13001,13080/13090/13100/13290/13540,Aix-En-Provence,"43.53607083783732, 5.398574445818331",0.55,0.74,2.11,0.98,NaN,1.96,5.32,1.96
4,13001009,2006-01-05,1.4,NaN,-3.4,223.0,6.2,2331.0,2.5,1.40,-7.6,NaN,9.6,1.4,1.2,9.8,110.0,2155.0,5.0,120.0,2157.0,NaN,NaN,69.0,2327.0,97.0,347.0,0.0,1212.0,6.4,87.0,NaN,NaN,0.1,0.1,13001,13080/13090/13100/13290/13540,Aix-En-Provence,"43.53607083783732, 5.398574445818331",0.55,0.74,2.12,0.98,NaN,1.92,5.36,1.83


In [241]:
df_meteo.rename({'DATE': 'Date'}, axis=1, inplace=True)
df_meteo['Date'] = pd.to_datetime(df_meteo['Date']).dt.normalize()

In [242]:
# df_meteo = df_meteo.rename(columns={
#     "année": "year",
#     "mois": "month",
#     "jour": "day"
# })

# # Étape 2 : Créer une colonne datetime à partir de ces 3 colonnes
# df_meteo["date"] = pd.to_datetime(df_meteo[["year", "month", "day"]])

df_meteo.head()

,POSTE,Date,RR,DRR,TN,HTN,TX,HTX,TM,TMNX,TNSOL,TN50,TAMPLI,TNTXM,FFM,FXI,DXI,HXI,FXY,DXY,HXY,FXI3S,HXI3S,UN,HUN,UX,HUX,DHUMI40,DHUMI80,TSVM,UM,ORAG,BRUME,ETPMON,ETPGRILLE,Code INSEE,Code Postal,ville,geo_point_2d,moyenne precipitations année,moyenne precipitations mois,moyenne evapotranspiration année,moyenne evapotranspiration mois,moyenne vitesse vent année,moyenne vitesse vent mois,moyenne temperature année,moyenne temperature mois
0,13001009,2006-01-01,0.2,NaN,3.7,823.0,10.2,1413.0,6.1,6.95,0.4,NaN,6.5,7.0,1.7,13.4,330.0,1157.0,6.3,320.0,1220.0,NaN,NaN,50.0,1432.0,98.0,7.0,0.0,1028.0,7.9,85.0,NaN,NaN,0.7,0.8,13001,13080/13090/13100/13290/13540,Aix-En-Provence,"43.53607083783732, 5.398574445818331",0.51,0.61,2.05,0.96,NaN,1.94,5.19,1.95
1,13001009,2006-01-02,0.0,NaN,2.8,604.0,9.9,1331.0,4.8,6.35,-0.9,NaN,7.1,6.4,2.5,12.8,330.0,1105.0,6.7,330.0,1107.0,NaN,NaN,57.0,1408.0,97.0,2348.0,0.0,814.0,6.7,79.0,NaN,NaN,1.2,1.2,13001,13080/13090/13100/13290/13540,Aix-En-Provence,"43.53607083783732, 5.398574445818331",0.51,0.75,2.07,0.95,NaN,1.91,5.23,1.96
2,13001009,2006-01-03,0.2,NaN,-2.6,610.0,10.1,1319.0,1.9,3.75,-7.2,NaN,12.7,3.8,0.8,6.4,330.0,1421.0,3.5,320.0,1430.0,NaN,NaN,39.0,1320.0,98.0,148.0,47.0,848.0,5.1,77.0,NaN,NaN,0.9,0.8,13001,13080/13090/13100/13290/13540,Aix-En-Provence,"43.53607083783732, 5.398574445818331",0.51,0.75,2.09,0.97,NaN,1.98,5.27,2.05
3,13001009,2006-01-04,0.0,NaN,-3.8,704.0,10.2,1247.0,1.2,3.20,-8.2,NaN,14.0,3.2,0.8,5.2,20.0,926.0,2.6,260.0,1348.0,NaN,NaN,40.0,1227.0,95.0,2152.0,6.0,856.0,4.9,77.0,NaN,NaN,0.9,0.7,13001,13080/13090/13100/13290/13540,Aix-En-Provence,"43.53607083783732, 5.398574445818331",0.55,0.74,2.11,0.98,NaN,1.96,5.32,1.96
4,13001009,2006-01-05,1.4,NaN,-3.4,223.0,6.2,2331.0,2.5,1.40,-7.6,NaN,9.6,1.4,1.2,9.8,110.0,2155.0,5.0,120.0,2157.0,NaN,NaN,69.0,2327.0,97.0,347.0,0.0,1212.0,6.4,87.0,NaN,NaN,0.1,0.1,13001,13080/13090/13100/13290/13540,Aix-En-Provence,"43.53607083783732, 5.398574445818331",0.55,0.74,2.12,0.98,NaN,1.92,5.36,1.83


In [249]:
df_meteo['Code INSEE'] = df_meteo['Code INSEE'].astype(str)

In [250]:
feux_ville['Date'] = pd.to_datetime(feux_ville['Date']).dt.normalize()



C:\Users\m_bar\AppData\Local\Temp\ipykernel_131148\1889379218.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  feux_ville['Date'] = pd.to_datetime(feux_ville['Date']).dt.normalize()


In [251]:

df_meteo['Code Postal'] = df_meteo['Code Postal'].astype(str)

### Merge Feux et météo

In [252]:
feux_ville = feux_ville.rename(columns={'Nom de la commune': 'ville'})
df= pd.merge(df_meteo, feux_ville, on=['Date', 'ville'], how='outer')

In [253]:
df.head()

,POSTE,Date,RR,DRR,TN,HTN,TX,HTX,TM,TMNX,TNSOL,TN50,TAMPLI,TNTXM,FFM,FXI,DXI,HXI,FXY,DXY,HXY,FXI3S,HXI3S,UN,HUN,UX,HUX,DHUMI40,DHUMI80,TSVM,UM,ORAG,BRUME,ETPMON,ETPGRILLE,Code INSEE_x,Code Postal,ville,geo_point_2d,moyenne precipitations année,moyenne precipitations mois,moyenne evapotranspiration année,moyenne evapotranspiration mois,moyenne vitesse vent année,moyenne vitesse vent mois,moyenne temperature année,moyenne temperature mois,Département,Code INSEE_y,latitude,longitude,Surface parcourue (m2),Surface forêt (m2),Surface maquis garrigues (m2),Autres surfaces naturelles hors forêt (m2),Surfaces agricoles (m2),Autres surfaces (m2),Surface autres terres boisées (m2),Surfaces non boisées naturelles (m2),Surfaces non boisées artificialisées (m2),Surfaces non boisées (m2),Type de peuplement,Nature,Feux
0,13001009.0,2006-01-01,0.2,NaN,3.7,823.0,10.2,1413.0,6.1,6.95,0.4,NaN,6.5,7.0,1.7,13.4,330.0,1157.0,6.3,320.0,1220.0,NaN,NaN,50.0,1432.0,98.0,7.0,0.0,1028.0,7.9,85.0,NaN,NaN,0.7,0.8,13001,13080/13090/13100/13290/13540,Aix-En-Provence,"43.53607083783732, 5.398574445818331",0.51,0.61,2.05,0.96,NaN,1.94,5.19,1.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13004003.0,2006-01-01,0.0,NaN,2.0,659.0,10.8,1327.0,7.1,6.40,-0.7,NaN,8.8,6.4,4.9,12.1,270.0,2150.0,8.5,280.0,1052.0,NaN,NaN,56.0,1314.0,98.0,3.0,0.0,583.0,7.8,78.0,NaN,NaN,1.0,1.0,13004,13200,Arles,"43.5468692377618, 4.6621564257363985",1.38,2.24,NaN,NaN,NaN,3.65,7.41,7.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13005003.0,2006-01-01,0.0,NaN,4.8,745.0,10.8,1415.0,7.8,7.80,2.0,NaN,6.0,7.8,3.9,16.0,340.0,1145.0,8.0,340.0,1130.0,NaN,NaN,51.0,1400.0,97.0,200.0,0.0,581.0,7.8,75.0,NaN,NaN,NaN,1.3,13005,13400,Aubagne,"43.29348437640274, 5.563312734769378",1.52,0.85,NaN,NaN,3.65,3.55,7.17,8.03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13022003.0,2006-01-01,0.0,NaN,5.2,804.0,9.9,1353.0,7.6,7.55,NaN,NaN,4.7,7.6,5.9,17.4,320.0,1205.0,10.0,320.0,1053.0,NaN,NaN,54.0,1354.0,92.0,215.0,0.0,493.0,7.8,75.0,NaN,NaN,NaN,1.4,13022,13260,Cassis,"43.223047006454145, 5.551178109907389",NaN,NaN,NaN,NaN,NaN,5.01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13030001.0,2006-01-01,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13030,13780,Cuges-Les-Pins,"43.28057170429038, 5.71024615752666",1.50,0.67,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [254]:
df['Feux'].value_counts()

Feux
1.0    4038
Name: count, dtype: int64

### Traitement des doublons

In [255]:
# df_clean = df.groupby(['ville', 'Date'], as_index=False).agg(lambda x: x.dropna().iloc[0] if not x.dropna().empty else None)

In [256]:
df['Feux'] = df['Feux'].fillna(0)
df['Feux'].value_counts()

Feux
0.0    95123
1.0     4038
Name: count, dtype: int64

In [257]:
# # # Supprime aussi les tabulations, retours chariots, etc.
# # df.columns = df.columns.str.replace(r"\s+", " ", regex=True).str.strip()
# # Filtrer les communes sans données météo mais AVEC des coordonnées valides
# df_manquante = df_manquante[df_manquante["latitude"].notna() & df_manquante["longitude"].notna()]

# # Pareil côté communes valides
# df_valide = df_valide[df_valide["latitude"].notna() & df_valide["longitude"].notna()]

# def trouver_commune_la_plus_proche(row, df_sources):
#     coord_source = (row["latitude"], row["longitude"])
#     distances = df_sources.apply(
#         lambda r: geodesic(coord_source, (r["latitude"], r["longitude"])).km, axis=1
#     )
#     return df_sources.loc[distances.idxmin()]

# colonnes_meteo = ["RR", "TX", "TM", "ETPMON", "UM",'RR','DRR','TN','HTN','TX','HTX','TM','TMNX','TNSOL','TN50','TAMPLI','TNTXM','FFM','FXI','DXI','HXI','FXY','DXY','HXY','FXI3S','HXI3S','UN','HUN','UX','HUX','DHUMI40','DHUMI80','TSVM','UM','ORAG','BRUME','ETPMON','ETPGRILLE','Code INSEE_x','Code Postal','Commune_x','geo_point_2d_x','moyenne precipitations année','moyenne precipitations mois','moyenne evapotranspiration année','moyenne evapotranspiration mois','moyenne vitesse vent année','moyenne vitesse vent mois','moyenne temperature année','moyenne temperature mois']  # ou toute autre liste pertinente


# # Boucle de remplissage
# for i, row in df_manquante.iterrows():
#     plus_proche = trouver_commune_la_plus_proche(row, df_valide)
#     for col in colonnes_meteo:
#         if pd.isna(row[col]):
#             df_manquante.at[i, col] = plus_proche[col]

# df_complet = pd.concat([df_valide, df_manquante], ignore_index=True)

In [258]:
df.to_csv(f"output/{path}_Dataset_modele.csv", index=False, sep=';')
